##Object Size Dataframe

Development Envrionment
<br/>xml Parsing
<br/>Object Size Calculation
<br/>Detailed Class Polygon Dataframe
<br/>Class Polygon Dataframe

#Development Envrionment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import random
import pandas as pd
from glob import glob
from pandas import option_context
from termcolor import colored
import matplotlib.pyplot as plt
from matplotlib import gridspec
from IPython.display import HTML
from IPython.display import display

In [ ]:
import xml.etree.ElementTree as ET
import xml.etree.ElementTree

#xml File Name List

In [ ]:
xml_folder_path = '/content/drive/MyDrive/Aiffelthon/eda/xml_data'
xml_folder_list = [xml_folder_path + '/train/train_part1_xml', xml_folder_path + '/train/train_part2_xml',
                   xml_folder_path + '/validation/vaildation_xml']

In [ ]:
def xml_file_list(xml_folder):
  label = glob(xml_folder+'/labelXml'+'/*')
  label = sorted(label, reverse=False)
  label = sorted(label, key=len)
  return label

In [ ]:
for i in xml_folder_list:
  if 'train_part1' in i:
    train_part1_label = xml_file_list(i)
  elif 'train_part2' in i:
    train_part2_label = xml_file_list(i)
  elif 'validation' in i:
    validation_label = xml_file_list(i)

In [ ]:
train_label = train_part1_label + train_part2_label

#xml Parsing

In [ ]:
tree = ET.parse(train_label[0])
root = tree.getroot()
ET.dump(tree)

<annotation>
	<source>
		<filename>0.tif</filename>
		<origin>GF2/GF3</origin>
	</source>
	<research>
		<version>1.0</version>
		<provider>FAIR1M</provider>
		<author>Cyber</author>
		<pluginname>FAIR1M</pluginname>
		<pluginclass>object detection</pluginclass>
		<time>2021-07-21</time>
	</research>
	<size>
		<width>1500</width>
		<height>1500</height>
		<depth>3</depth>
	</size>
	<objects>
		<object>
			<coordinate>pixel</coordinate>
			<type>rectangle</type>
			<description>None</description>
			<possibleresult>
				<name>Liquid Cargo Ship</name>
			</possibleresult>
			<points>
				<point>1275.000000,458.000000</point>
				<point>1494.000000,88.000000</point>
				<point>1417.000000,43.000000</point>
				<point>1199.000000,414.000000</point>
				<point>1275.000000,458.000000</point>
			</points>
		</object>
		<object>
			<coordinate>pixel</coordinate>
			<type>rectangle</type>
			<description>None</description>
			<possibleresult>
				<name>Passenger Ship</name>
			</possibleresult>


#Object Size Calculation

In [ ]:
tree = ET.parse(train_label[0])
root = tree.getroot()
print(root[0][0].text)
print()

class_name_list = []
file_point_list = []

for i in root[3]:
  for child in i[3]:
    class_name_list.append(child.text)
    class_point_list = []
    for j in range(len(i[4])):
      class_point_tuple = []
      points = i[4][j].text.split(',')
      for point in points:
        point = float(point)
        class_point_tuple.append(point)
      class_point_tuple = tuple(class_point_tuple)
      class_point_list.append(class_point_tuple)
    file_point_list.append(class_point_list)     

for i, j in zip(class_name_list, file_point_list):
  print(i, j)

0.tif

Liquid Cargo Ship [(1275.0, 458.0), (1494.0, 88.0), (1417.0, 43.0), (1199.0, 414.0), (1275.0, 458.0)]
Passenger Ship [(630.0, 403.0), (586.0, 385.0), (582.0, 397.0), (625.0, 414.0), (630.0, 403.0)]
Dry Cargo Ship [(591.0, 1531.0), (761.0, 1377.0), (718.0, 1332.0), (549.0, 1486.0), (591.0, 1531.0)]


[TUTORIAL 2: CUSTOMIZE DATASETS](https://mmrotate.readthedocs.io/en/latest/tutorials/customize_dataset.html)

The annotation txt files in DOTA format:
<br/>Polygons with format (x1, y1, x2, y2, x3, y3, x4, y4)

[How to calculate area of a polygon with latitude and longitude?](https://stackoverflow.com/questions/45733838/how-to-calculate-area-of-a-polygon-with-latitude-and-longitude)

In [ ]:
import tifffile as tiff
from shapely.geometry import Polygon

In [ ]:
object_polygon_list = []
object_size_list = []

for points in file_point_list:
  object_polygon = Polygon(points)
  object_size = object_polygon.area
  object_polygon_list.append(object_polygon)
  object_size_list.append(object_size)

In [ ]:
for i, j, k in zip(class_name_list, file_point_list, object_length_list):
  print(i, j, k)

Liquid Cargo Ship [(1275.0, 458.0), (1494.0, 88.0), (1417.0, 43.0), (1199.0, 414.0), (1275.0, 458.0)] 38066.5
Passenger Ship [(630.0, 403.0), (586.0, 385.0), (582.0, 397.0), (625.0, 414.0), (630.0, 403.0)] 579.0
Dry Cargo Ship [(591.0, 1531.0), (761.0, 1377.0), (718.0, 1332.0), (549.0, 1486.0), (591.0, 1531.0)] 14172.5


#Detailed Class Polygon Dataframe

In [ ]:
def detailed_class_polygon_df_init():
  detailed_class_list = ['Dump Truck', 'Cargo Truck', 'Truck Tractor', 'Tractor', 'Excavator', 'Trailer', 'Van', 'Small Car', 'Bus', 'other-vehicle',
                         'Dry Cargo Ship', 'Liquid Cargo Ship', 'Motorboat', 'Fishing Boat', 'Tugboat', 'Engineering Ship', 'Warship', 'Passenger Ship', 'other-ship',
                         'A220', 'A321', 'A330',  'A350', 'Boeing737', 'Boeing747', 'Boeing777', 'Boeing787', 'ARJ21', 'C919', 'other-airplane']

  polygon_df = pd.DataFrame({'file':['a'], 'object size':[0.0], 
                                    'point':[[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]]})
  # vehicle
  dump_truck_polygon_df = polygon_df.copy(); cargo_truck_polygon_df = polygon_df.copy()
  truck_tractor_polygon_df = polygon_df.copy(); tractor_polygon_df = polygon_df.copy()
  excavator_polygon_df = polygon_df.copy(); trailer_polygon_df = polygon_df.copy(); van_polygon_df = polygon_df.copy()
  small_car_polygon_df = polygon_df.copy(); bus_polygon_df = polygon_df.copy(); other_vehicle_polygon_df = polygon_df.copy()

  # ship
  dry_cargo_ship_polygon_df = polygon_df.copy(); liquid_cargo_ship_polygon_df = polygon_df.copy(); motorboat_polygon_df = polygon_df.copy()
  fishing_boat_polygon_df = polygon_df.copy(); tug_boat_polygon_df = polygon_df.copy(); engineering_ship_polygon_df = polygon_df.copy()
  warship_polygon_df = polygon_df.copy(); passenger_ship_polygon_df = polygon_df.copy(); other_ship_polygon_df = polygon_df.copy()

  # airplane
  a220_polygon_df = polygon_df.copy(); a321_polygon_df = polygon_df.copy();
  a330_polygon_df = polygon_df.copy(); a350_polygon_df = polygon_df.copy(); boeing737_polygon_df = polygon_df.copy();
  boeing747_polygon_df = polygon_df.copy(); boeing777_polygon_df = polygon_df.copy(); boeing787_polygon_df = polygon_df.copy();
  arj21_polygon_df = polygon_df.copy(); c919_polygon_df = polygon_df.copy(); other_airplane_polygon_df = polygon_df.copy()

  # detailed class
  detailed_class_polygon_df_list = [dump_truck_polygon_df, cargo_truck_polygon_df, truck_tractor_polygon_df, tractor_polygon_df, excavator_polygon_df,
                                  trailer_polygon_df, van_polygon_df, small_car_polygon_df, bus_polygon_df, other_vehicle_polygon_df, 
                                  dry_cargo_ship_polygon_df, liquid_cargo_ship_polygon_df, motorboat_polygon_df, fishing_boat_polygon_df,
                                  tug_boat_polygon_df, engineering_ship_polygon_df, warship_polygon_df, passenger_ship_polygon_df, other_ship_polygon_df,
                                  a220_polygon_df, a321_polygon_df, a330_polygon_df, a350_polygon_df, boeing737_polygon_df, boeing747_polygon_df,
                                  boeing777_polygon_df, boeing787_polygon_df, arj21_polygon_df, c919_polygon_df, other_airplane_polygon_df]

  return detailed_class_list, dump_truck_polygon_df, cargo_truck_polygon_df, truck_tractor_polygon_df, tractor_polygon_df, excavator_polygon_df, \
  trailer_polygon_df, van_polygon_df, small_car_polygon_df, bus_polygon_df, other_vehicle_polygon_df, dry_cargo_ship_polygon_df, \
  liquid_cargo_ship_polygon_df, motorboat_polygon_df, fishing_boat_polygon_df, tug_boat_polygon_df, engineering_ship_polygon_df, warship_polygon_df, \
  passenger_ship_polygon_df, other_ship_polygon_df,a220_polygon_df, a321_polygon_df, a330_polygon_df, a350_polygon_df, boeing737_polygon_df, \
  boeing747_polygon_df, boeing777_polygon_df, boeing787_polygon_df, arj21_polygon_df, c919_polygon_df, other_airplane_polygon_df, detailed_class_polygon_df_list

In [ ]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [ ]:
def detailed_class_polygon(label, class_list, polygon_df_list):
  file_name_list = []
  class_name_list = []
  file_point_list = []

  for i in label:
    tree = ET.parse(i)
    root = tree.getroot()
    
    for i in root[3]:
      for child in i[3]:
        class_name_list.append(child.text)
        class_point_list = []
        for j in range(0, 4):
          class_point_tuple = []
          points = i[4][j].text.split(',')
          for point in points:
            point = float(point)
            class_point_tuple.append(point)
          class_point_tuple = tuple(class_point_tuple)
          class_point_list.append(class_point_tuple)
        file_point_list.append(class_point_list)     
        file_name_list.append(root[0][0].text)

  object_size_list = []
  for points in file_point_list:
    object_polygon = Polygon(points)
    object_size = object_polygon.area
    object_size_list.append(object_size)
  
  count = -1
  for i, j, k, l in zip(file_name_list, class_name_list, object_size_list, file_point_list):
    count += 1
    for m in class_list:
      list_index = class_list.index(m)
      if j == class_list[list_index]:
        polygon_df_list[list_index].loc[count] = [i, k, l]

  for i in range(len(polygon_df_list)):
    df_name = [k for k,v in globals().items() if id(v) == id(polygon_df_list[i])][0]
    if 'train' in namestr(label, globals())[0]:
      df_name = 'train_' + df_name 
    elif 'validation' in namestr(label, globals())[0]:
      df_name = 'validation_' + df_name 
    globals()[df_name] = polygon_df_list[i][1:]
    globals()[df_name].to_csv('/content/drive/MyDrive/Aiffelthon/eda/object_size/' + df_name.split("_")[0] +'_detailed_class_polygon_df/' + df_name + '.csv', index = False)

namestr(label, globals())[ 0 ]이거나 namestr(label, globals())[ 1 ]이다.
<br/>train_label, validation_label에 따라 다르다.

In [ ]:
detailed_class_list, dump_truck_polygon_df, cargo_truck_polygon_df, truck_tractor_polygon_df, tractor_polygon_df, excavator_polygon_df, \
trailer_polygon_df, van_polygon_df, small_car_polygon_df, bus_polygon_df, other_vehicle_polygon_df, dry_cargo_ship_polygon_df, \
liquid_cargo_ship_polygon_df, motorboat_polygon_df, fishing_boat_polygon_df, tug_boat_polygon_df, engineering_ship_polygon_df, warship_polygon_df, \
passenger_ship_polygon_df, other_ship_polygon_df,a220_polygon_df, a321_polygon_df, a330_polygon_df, a350_polygon_df, boeing737_polygon_df, \
boeing747_polygon_df, boeing777_polygon_df, boeing787_polygon_df, arj21_polygon_df, c919_polygon_df, other_airplane_polygon_df, detailed_class_polygon_df_list \
= detailed_class_polygon_df_init()

detailed_class_polygon(train_label, detailed_class_list, detailed_class_polygon_df_list)

In [ ]:
detailed_class_list, dump_truck_polygon_df, cargo_truck_polygon_df, truck_tractor_polygon_df, tractor_polygon_df, excavator_polygon_df, \
trailer_polygon_df, van_polygon_df, small_car_polygon_df, bus_polygon_df, other_vehicle_polygon_df, dry_cargo_ship_polygon_df, \
liquid_cargo_ship_polygon_df, motorboat_polygon_df, fishing_boat_polygon_df, tug_boat_polygon_df, engineering_ship_polygon_df, warship_polygon_df, \
passenger_ship_polygon_df, other_ship_polygon_df,a220_polygon_df, a321_polygon_df, a330_polygon_df, a350_polygon_df, boeing737_polygon_df, \
boeing747_polygon_df, boeing777_polygon_df, boeing787_polygon_df, arj21_polygon_df, c919_polygon_df, other_airplane_polygon_df, detailed_class_polygon_df_list \
= detailed_class_polygon_df_init()

detailed_class_polygon(validation_label, detailed_class_list, detailed_class_polygon_df_list)

In [ ]:
csv_files = glob('/content/drive/MyDrive/Aiffelthon/eda/object_size/train_detailed_class_polygon_df/' + '*.csv')
  
count = 0
random_numbers = random.sample(range(1, len(csv_files)), 3)
for f in csv_files:
    count += 1
    for random_number in random_numbers:
      if count == random_number:
        df = pd.read_csv(f)
        file_name = ' '.join(s for s in f.split("/")[-1][:-4].split("_")[1:-2])
        if file_name == 'an':
          print(f.split("/")[-1].strip(".csv"))
        print(colored(file_name, attrs=['bold']))
        print()
        pd.set_option('display.max_rows', 10)
        with option_context('display.max_colwidth', None):
          display(df)
          print("\n" * 2)

truck tractor



,file,object size,point
0,21.tif,219.890501,"[(539.0, 656.999939), (538.109741, 665.012146), (511.0, 661.999939), (511.890259, 653.987732)]"
1,29.tif,119.000000,"[(13.0, 434.0), (-1.0, 432.0), (0.0, 424.0), (15.0, 426.0)]"
2,188.tif,344.898179,"[(729.000061, 650.999939), (718.011536, 651.354431), (717.000061, 619.999939), (727.988586, 619.645447)]"
3,216.tif,115.484256,"[(-7e-06, 917.947388), (-4.538147, 912.55835), (7.999978, 901.999878), (12.538118, 907.388916)]"
4,216.tif,151.000548,"[(26.408487, 887.405762), (15.408487, 903.405762), (8.999998, 898.999878), (19.999998, 882.999878)]"
...,...,...,...
918,15830.tif,416.000000,"[(91.0, 447.0), (77.0, 449.0), (72.0, 420.0), (86.0, 418.0)]"
919,16119.tif,56.000000,"[(132.0, 44.0), (124.0, 44.0), (124.0, 37.0), (132.0, 37.0)]"
920,16130.tif,436.500000,"[(44.0, 782.0), (43.0, 792.0), (0.0, 789.0), (0.0, 779.0)]"
921,16209.tif,375.000000,"[(331.0, 546.0), (290.0, 568.0), (286.0, 561.0), (327.0, 539.0)]"





trailer



,file,object size,point
0,6.tif,802.000000,"[(802.0, 547.0), (763.0, 596.0), (753.0, 588.0), (792.0, 539.0)]"
1,6.tif,634.000000,"[(755.0, 604.0), (716.0, 654.0), (708.0, 648.0), (747.0, 598.0)]"
2,6.tif,717.000000,"[(708.0, 665.0), (668.0, 715.0), (660.0, 707.0), (699.0, 658.0)]"
3,6.tif,669.000000,"[(665.0, 719.0), (628.0, 765.0), (620.0, 758.0), (656.0, 711.0)]"
4,39.tif,299.000736,"[(681.000061, 563.0), (679.000061, 590.0), (667.986389, 589.184204), (669.986389, 562.184204)]"
...,...,...,...
584,16345.tif,142.500000,"[(711.0, 128.0), (692.0, 127.0), (692.0, 120.0), (711.0, 120.0)]"
585,16347.tif,463.000000,"[(47.0, 742.0), (38.0, 749.0), (13.0, 717.0), (22.0, 710.0)]"
586,16395.tif,714.000000,"[(597.0, 160.0), (593.0, 171.0), (537.0, 148.0), (542.0, 137.0)]"
587,16484.tif,493.000000,"[(705.0, 163.0), (699.0, 200.0), (686.0, 198.0), (692.0, 161.0)]"





a321



,file,object size,point
0,32.tif,2615.000000,"[(-8.0, 407.0), (35.0, 444.0), (65.0, 409.0), (22.0, 372.0)]"
1,52.tif,2568.000000,"[(124.0, 965.0), (163.0, 922.0), (128.0, 893.0), (91.0, 934.0)]"
2,54.tif,6474.000000,"[(548.0, 626.0), (461.0, 661.0), (486.0, 725.0), (573.0, 691.0)]"
3,55.tif,2666.000000,"[(27.0, 1009.0), (66.0, 965.0), (32.0, 935.0), (-7.0, 979.0)]"
4,55.tif,2382.000000,"[(155.0, 453.0), (207.0, 436.0), (194.0, 395.0), (141.0, 412.0)]"
...,...,...,...
2500,16466.tif,2390.000000,"[(102.0, 637.0), (142.0, 674.0), (172.0, 642.0), (132.0, 605.0)]"
2501,16471.tif,2711.999325,"[(256.000031, 110.999985), (211.000031, 113.999985), (207.000031, 54.0), (252.000031, 51.0)]"
2502,16480.tif,2233.000000,"[(455.0, 865.0), (416.0, 834.0), (388.0, 869.0), (427.0, 900.0)]"
2503,16480.tif,2434.500000,"[(863.0, 940.0), (906.0, 976.0), (934.0, 942.0), (891.0, 907.0)]"


#Class Polygon Dataframe

In [ ]:
def class_init():
  vehicle_class_list = ['Dump Truck', 'Cargo Truck', 'Truck Tractor', 'Tractor','Excavator', 'Trailer', 'Van', 'Small Car', 'Bus', 'other-vehicle']
  ship_class_list = ['Dry Cargo Ship', 'Liquid Cargo Ship', 'Motorboat', 'Fishing Boat', 'Tugboat', 'Engineering Ship', 'Warship', 'Passenger Ship', 'other-ship']
  airplane_class_list = ['A220', 'A321', 'A330',  'A350', 'Boeing737', 'Boeing747', 'Boeing777', 'Boeing787', 'ARJ21', 'C919', 'other-airplane']
  
  class_list = [vehicle_class_list, ship_class_list, airplane_class_list]

  polygon_df = pd.DataFrame({'file':['a'], 'object size':[0.0], 
                                    'point':[[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]]})
  
  vehicle_polygon_df = polygon_df.copy(); 
  ship_polygon_df = polygon_df.copy();
  airplane_polygon_df = polygon_df.copy();

  class_polygon_df_list = [vehicle_polygon_df, ship_polygon_df, airplane_polygon_df]

                                  
  return vehicle_class_list, ship_class_list, airplane_class_list, class_list, \
  vehicle_polygon_df, ship_polygon_df, airplane_polygon_df, class_polygon_df_list

In [ ]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [ ]:
def class_polygon(label, class_list, polygon_df_list):
  file_name_list = []
  class_name_list = []
  file_point_list = []

  for i in label:
    tree = ET.parse(i)
    root = tree.getroot()
    for class_name in class_list:
      list_index = class_list.index(class_name)
      for detailed_class_name in class_name: 
        for i in root[3]:
          for child in i[3]:
            if child.text == detailed_class_name:
              class_name_list.append(class_name)
              class_point_list = []
              for j in range(0, 4):
                class_point_tuple = []
                points = i[4][j].text.split(',')
                for point in points:
                  point = float(point)
                  class_point_tuple.append(point)
                class_point_tuple = tuple(class_point_tuple)
                class_point_list.append(class_point_tuple)
              file_point_list.append(class_point_list)     
              file_name_list.append(root[0][0].text)

  object_size_list = []
  for points in file_point_list:
    object_polygon = Polygon(points)
    object_size = object_polygon.area
    object_size_list.append(object_size)
  
  count = -1
  for i, j, k, l in zip(file_name_list, class_name_list, object_size_list, file_point_list):
    count += 1
    for m in class_list:
      list_index = class_list.index(m)
      if j == class_list[list_index]:
        polygon_df_list[list_index].loc[count] = [i, k, l]

  for i in range(len(polygon_df_list)):
    df_name = [k for k,v in globals().items() if id(v) == id(polygon_df_list[i])][0]
    if 'train' in namestr(label, globals())[0]:
      df_name = 'train_' + df_name 
    elif 'validation' in namestr(label, globals())[0]:
      df_name = 'validation_' + df_name 
    globals()[df_name] = polygon_df_list[i][1:]
    globals()[df_name].to_csv('/content/drive/MyDrive/Aiffelthon/eda/object_size/' + df_name.split("_")[0] +'_class_polygon_df/' + df_name + '.csv', index = False)

namestr(label, globals())[ 0 ]이거나 namestr(label, globals())[ 1 ]이다.
<br/>train_label, validation_label에 따라 다르다.

In [ ]:
vehicle_class_list, ship_class_list, airplane_class_list, class_list, \
vehicle_polygon_df, ship_polygon_df, airplane_polygon_df, class_polygon_df_list \
= class_init()

class_polygon(train_label, class_list, class_polygon_df_list)

In [ ]:
vehicle_class_list, ship_class_list, airplane_class_list, class_list, \
vehicle_polygon_df, ship_polygon_df, airplane_polygon_df, class_polygon_df_list \
= class_init()

class_polygon(validation_label, class_list, class_polygon_df_list)

In [ ]:
csv_files = glob('/content/drive/MyDrive/Aiffelthon/eda/object_size/train_class_polygon_df/' + '*.csv')
  
for f in csv_files:
  df = pd.read_csv(f)
  file_name = ' '.join(s for s in f.split("/")[-1][:-4].split("_")[1:-2])
  if file_name == 'an':
    print(f.split("/")[-1].strip(".csv"))
  print(colored(file_name, attrs=['bold']))
  print()
  pd.set_option('display.max_rows', 10)
  with option_context('display.max_colwidth', None):
    display(df)
    print("\n" * 2)

vehicle



,file,object size,point
0,1.tif,110.000000,"[(43.000004, 953.000061), (38.000004, 958.000061), (27.000004, 947.000061), (32.000004, 942.000061)]"
1,1.tif,192.398082,"[(163.124451, 965.919556), (146.000061, 977.0), (140.875671, 969.080444), (158.000061, 958.0)]"
2,1.tif,294.822077,"[(511.161621, 786.105713), (505.999939, 794.0), (479.838257, 776.894287), (484.999939, 769.0)]"
3,1.tif,298.001460,"[(475.361877, 766.298828), (469.000061, 774.0), (446.000061, 755.0), (452.361877, 747.298828)]"
4,1.tif,368.580762,"[(364.086121, 761.098999), (355.0, 769.0), (334.913879, 745.901001), (344.0, 738.0)]"
...,...,...,...
317626,16484.tif,141.000000,"[(512.0, 454.0), (499.0, 464.0), (494.0, 457.0), (507.0, 447.0)]"
317627,16484.tif,155.000000,"[(625.0, 428.0), (623.0, 445.0), (614.0, 444.0), (616.0, 427.0)]"
317628,16484.tif,201.000000,"[(438.0, 203.0), (436.0, 224.0), (427.0, 223.0), (428.0, 202.0)]"
317629,16484.tif,153.500000,"[(450.0, 204.0), (448.0, 220.0), (439.0, 219.0), (440.0, 203.0)]"





ship



,file,object size,point
0,0.tif,38066.5,"[(1275.0, 458.0), (1494.0, 88.0), (1417.0, 43.0), (1199.0, 414.0)]"
1,0.tif,579.0,"[(630.0, 403.0), (586.0, 385.0), (582.0, 397.0), (625.0, 414.0)]"
2,6.tif,313.5,"[(35.0, 543.0), (29.0, 550.0), (2.0, 525.0), (7.0, 519.0)]"
3,6.tif,218.0,"[(8.0, 664.0), (-7.0, 650.0), (0.0, 642.0), (15.0, 656.0)]"
4,6.tif,214.0,"[(20.0, 708.0), (0.0, 725.0), (0.0, 712.0), (13.0, 701.0)]"
...,...,...,...
31456,16470.tif,113.5,"[(199.0, 430.0), (197.0, 453.0), (192.0, 452.0), (194.0, 430.0)]"
31457,16470.tif,193.0,"[(215.0, 452.0), (203.0, 471.0), (196.0, 466.0), (208.0, 447.0)]"
31458,16477.tif,1302.0,"[(91.0, 95.0), (76.0, 164.0), (58.0, 160.0), (73.0, 91.0)]"
31459,16477.tif,1218.5,"[(70.0, 170.0), (58.0, 233.0), (39.0, 230.0), (52.0, 166.0)]"





airplane



,file,object size,point
0,2.tif,2005.5,"[(900.0, 461.0), (901.0, 506.0), (946.0, 504.0), (944.0, 459.0)]"
1,2.tif,1937.0,"[(972.0, 309.0), (1000.0, 275.0), (967.0, 247.0), (938.0, 281.0)]"
2,2.tif,2057.0,"[(614.0, 861.0), (644.0, 825.0), (612.0, 797.0), (580.0, 834.0)]"
3,4.tif,2651.0,"[(283.0, 447.0), (234.0, 458.0), (245.0, 509.0), (295.0, 498.0)]"
4,4.tif,2072.0,"[(950.0, 281.0), (921.0, 317.0), (956.0, 345.0), (985.0, 309.0)]"
...,...,...,...
30319,16485.tif,1810.5,"[(104.0, 332.0), (78.0, 364.0), (44.0, 335.0), (71.0, 304.0)]"
30320,16486.tif,6380.0,"[(177.0, 830.0), (238.0, 773.0), (186.0, 717.0), (125.0, 774.0)]"
30321,16487.tif,2000.5,"[(773.0, 273.0), (745.0, 307.0), (710.0, 277.0), (737.0, 244.0)]"
30322,16487.tif,2322.0,"[(305.0, 540.0), (277.0, 572.0), (236.0, 536.0), (265.0, 504.0)]"
